![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx.ai and LangChain Agents to perform E-commerce Analytics

<img src="https://github.com/user-attachments/assets/66ba8ea8-6481-41c6-aeaa-9a952574236e" width=50%></img>

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate usage of LangChain Agents with watsonx.ai models.

Some familiarity with Python is helpful. This notebook uses Python 3.10.


## Learning goal

The purpose of this notebook is to demonstrate how to use large language models, e.g. `mistralai/mistral-large`, to create Structured Chat LangChain Agent with memory, where an Agent may perform sequence of actions based on the model reasoning and assures that LLM uses chat memory to provide the most accurate answer on the user question. The Agent can also use various tools such as SQL or RAG retriever to get the relevant information from structured data sources such as SQL tables or unstrustured data sources such as Vectorstore. 


## Contents

This notebook contains the following parts:

- [Pre requisites](#)
- [Setup](#setup)
- [LangChain integration](#watsonxllm)
- [Data Preparation](#data_preparation)
- [Prompt template and custom tools](#prompt_template)
- [LangChain Agent](#agent)
- [Example of Structured Chat](#chat)
- [Summary](#summary)

<a id="pre_requisites"></a>
## Pre requisites

You need to have a Db2 Database as a pre-requisite. Create tables in the Db2 Database with the ecommerce datasets provided in the watsonx project. Your Db2 should have these tables:

- customer_purchase_history
- product_inventory
- sales_data

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the `datasets` and dependecies

In [ ]:
!pip install -U "ibm-watsonx-ai==1.1.2" | tail -n 1
!pip install -U "pydantic==2.8.2" | tail -n 1
!pip install -U "langchain==0.2.12" | tail -n 1
!pip install -U "langchain-ibm==0.1.11" | tail -n 1
!pip install -U "langchain-core==0.2.28" | tail -n 1
!pip install -U "langchain-community==0.2.11" | tail -n 1
!pip install -U "langchain_huggingface==0.0.3" | tail -n 1
!pip install -U "sqlalchemy==1.4.47" | tail -n 1
!pip install -U "pymilvus==2.4.4" | tail -n 1
!pip install -U "PyMuPDF==1.24.9" | tail -n 1

**Note: Restart the kernel after installing the dependencies** 

### Defining the WML credentials
This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentation</a>.

In [1]:
import getpass

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

Please enter your WML api key (hit enter): ········


### Defining the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [2]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="watsonxllm"></a>
## Initialize `WatsonxLLM` instance

#### Set model parameters

To set the model parameters we use `GenTextParamsMetaNames` metanames from `ibm-watsonx-ai`.

In [3]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames
from langchain_ibm import WatsonxLLM

Based on the performed experiments we found that the following parameters work well with the LangChain Agents created in the further section.

In [4]:
parameters = {
    GenTextParamsMetaNames.MAX_NEW_TOKENS: 250,
    GenTextParamsMetaNames.MIN_NEW_TOKENS: 20,
    GenTextParamsMetaNames.DECODING_METHOD: 'greedy',
}

Furthermore, you need to specify `model_id` that will be used for inferencing. In the following notebook we use `mistralai/mistral-large`.

Based on the performed experiments we found that the following models work well with LangChain Agents:

- mistralai/mistral-large
- meta-llama/llama-3-405b-instruct
- meta-llama/llama-3-70b-instruct
- meta-llama/llama-2-70b-chat
- mistralai/mixtral-8x7b-instruct-v01

In [5]:
llm = WatsonxLLM(
    url=credentials.get('url'),
    apikey=credentials.get('apikey'),
    project_id=project_id,
    model_id="mistralai/mistral-large",
    params=parameters
)

#### Test the model

In [6]:
response = llm.invoke('What is a generative AI? Respond concisely.')
print(response)



Generative AI is a type of artificial intelligence that creates new content, such as images, music, or text. It learns patterns from input data and generates new, likely outputs. Examples include DALL-E for images and ChatGPT for text.


<a id="data_preparation"></a>
## Data Preparation
### Create SQL Tables

In this section we load the structured data into SQL tables.

- **Structured data**: SQL Database with three tables
  - **Sales Data**
  - **Product Inventory**
  - **Customer Purchase History**

In [7]:
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import ibm_db
import ibm_db_sa
from langchain.sql_database import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit

In [8]:
import contextlib
from tempfile import TemporaryFile
from tabulate import tabulate
import os

### Add your Db2 Credentials in the cell below

In [9]:
@contextlib.contextmanager
def get_ecommerce_db():
    # Define your database connection parameters
    user = ""
    pwd = ""
    host = ""
    port = ""
    db_name = "bludb"
    
    connection_string = f'db2://{user}:{pwd}@{host}:{port}/{db_name};SECURITY=SSL;PROTOCOL=TCPIP;'
    
    # Create the SQLAlchemy engine
    engine = None
    try:
        engine = create_engine(connection_string)
        
        # Initialize the SQLDatabase object
        db = SQLDatabase(engine)
        yield db

    except SQLAlchemyError as e:
        print(f"An error occurred while connecting to the database: {e}")
        yield None

    finally:
        if engine:
            # Ensure the engine is disposed of properly
            engine.dispose()

### Create Vectorstore

In this section we load the unstructured data into vectorstore.

- **Unstructured data**: Text file with two sections
  -  **Customer Reviews**
  -  **Social Media Mentions**

### Add data from watsonx project 

1. Click on the below cell to bring the cursor to the cell.
1. Click on the **Code Snippets** button on top right corner of the watson studio.
1. Click on **Select data from project**.
1. Select the category as **Data asset** and select the **reviews.txt** file.
1. Select the load as **StreamingBody object** and click on **Insert code to cell**

In [10]:
# Add review.txt streaming body object here

>Note: Make sure the streaming body variable name is `streaming_body_1`

In [11]:
file_text = streaming_body_1.read().decode('utf-8')

In [12]:
with open('reviews.txt', 'w') as ffile:
    ffile.write(file_text)

<a id="Define RAG Pipeline"></a>
## Define RAG Pipeline
### Load, Split, Embedd, Create collection

In [13]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tqdm import tqdm

In [14]:
from pymilvus import MilvusClient

In [15]:
def rag_load(collection_name) -> MilvusClient:
    loader = TextLoader('reviews.txt')
    docs = loader.load()
    print("Number of pages:", len(docs))
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, is_separator_regex = False)
    splits = text_splitter.split_documents(docs)
    print("Number of splits:", len(splits))
    milvus_client = MilvusClient("./milvus_rag.db")
    if milvus_client.has_collection(collection_name):
        milvus_client.drop_collection(collection_name)
    milvus_client.create_collection(
        collection_name=collection_name,
        dimension=768,
        metric_type="IP",  # Inner product distance
        consistency_level="Strong",  # Strong consistency level
    )
    data = []
    for i, line in enumerate(tqdm(splits, desc="Creating embeddings please wait...")):
        data.append({"id": i, "vector": embeddings.embed_query(line.page_content), "text": line.page_content})
    milvus_client.insert(collection_name=collection_name, data=data)
    return milvus_client

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings

In [17]:
embeddings = HuggingFaceEmbeddings()
collection_name = 'my_rag_collection'
milvus_client = rag_load(collection_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Number of pages: 1
Number of splits: 4


Creating embeddings please wait...: 100%|██████████| 4/4 [00:01<00:00,  2.18it/s]


### Create Retriever

In [18]:
def rag_retriever(query: str) -> str:
    result = ""
    search_res = milvus_client.search(
        collection_name=collection_name,
        data=[
            embeddings.embed_query(query)
        ],
        limit=2,
        search_params={"metric_type": "IP", "params": {}},
        output_fields=["text"],
    )
    retrieved_lines_with_distances = [
        (res["entity"]["text"], res["distance"]) for res in search_res[0]
    ]
    for item in retrieved_lines_with_distances:
        result += "[Document]\n"
        result += item[0] + "\n"
        result += "[End]\n\n"
    return result.strip()

#### Test RAG Retriever 

In [19]:
print(rag_retriever("What are the customer reviews?"))

[Document]
## Customer Reviews

ReviewID: R001
Text: "The iPhone is fantastic! Great build quality and very useful."
ProductID: P001
Sentiment: Positive
Name: Alice

ReviewID: R002
Text: "The iPad is okay, but a bit overpriced for what it offers."
ProductID: P002
Sentiment: Neutral
Name: Bob

ReviewID: R003
Text: "The Mac exceeded my expectations. Highly recommend it!"
ProductID: P003
Sentiment: Positive
Name: Charlie

ReviewID: R004
Text: "The AirPods are not as good as I expected. They broke after a week."
ProductID: P004
Sentiment: Negative
Name: Alice

ReviewID: R005
Text: "The iPhone is great, but could be a bit cheaper."
ProductID: P001
Sentiment: Positive
Name: David

ReviewID: R006
Text: "The iPad is very useful for my daily tasks."
ProductID: P002
Sentiment: Positive
Name: Bob

ReviewID: R007
Text: "The Mac is simply the best computer I have ever used."
ProductID: P003
Sentiment: Positive
Name: Charlie
[End]

[Document]
ReviewID: R007
Text: "The Mac is simply the best computer

<a id="prompt_template"></a>
## Prompt template preparation and tools
### Create Prompt Template

A structured chat prompt template is composed of several sections: system prompt, chat history and user input. The following prompt template was prepared based on the LangChain [documentation](https://python.langchain.com/docs/modules/agents/agent_types/structured_chat) and adjusted to work with the chat models available on watsonx.ai.

In [20]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import MessagesPlaceholder

In [21]:
system_prompt = """Respond to the human as helpfully and accurately as possible. You have access to the following tools:
{tools}
Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
Valid "action" values: "Final Answer" or {tool_names}
Provide only ONE action per $JSON_BLOB, as shown:
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```
Follow this format:
Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
Begin! Reminder to ALWAYS respond with a valid json blob of a single action.
Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation.
Reminder to ALWAYS respond with a valid JSON blob of a single action. Use tools only if necessary.
Reminder to ALWAYS list the available tables before executing any SQL query on a table.
Reminder to ALWAYS check for relationship between tables if column name is not found.
"""

In [22]:
user_prompt = """{input}
{agent_scratchpad}
(reminder to respond in a JSON blob no matter what)"""

To create the final structured chat prompt template one can use initialization class method from `ChatPromptTemplate` .

In [23]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", system_prompt
        ),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        ("user", user_prompt),
    ]
)

### Prepare tools

In this section we prepare tools (python functions) that the AI assistant will have access to.

In [24]:
from langchain.agents import tool
from langchain.tools import Tool

In [25]:
@tool("milvus_retriever")
def milvus_retriever(query: str) -> str:
    """
    Returns the matching occurance of the query from the vectorstore.

    Args:
        query: The query string to match in the vectorstore.

    Returns:
        str: Nearest occurances of the query string.
    """
    formatted_response = rag_retriever(query)
    
    return formatted_response

In [26]:
rag_tool = Tool(
    name="milvus_retriever",
    func=milvus_retriever,
    description="Search for information in the vector database.",
)

In [27]:
with get_ecommerce_db() as db:
    if db is not None:
        sql_toolkit = SQLDatabaseToolkit(db=db, llm=llm)
        tools_chat = sql_toolkit.get_tools()
    else:
        print("Failed to connect to the database.")

In [28]:
tools_chat.append(rag_tool)

Furthermore, we define custom `CustomJSONAgentOutputParser` because sometimes model may return response with additional tokens before action JSON blob. 

In [29]:
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain_core.agents import AgentAction, AgentFinish

In [30]:
class CustomJSONAgentOutputParser(JSONAgentOutputParser):
    def parse(self, text: str) -> AgentAction | AgentFinish:
        i = text.find("{")

        # Sometimes model may return additional tokens before json blob
        text = text[i:]
        if "```" not in text:
            text = "\n```\n" + text
        return super().parse(text + "\n```\nObservation:")

<a id="agent"></a>
## Initialize an Agent

Following section is the central part of the presented notebook. Here, we construct LangChain Agent and initialise `AgentExecutor` which manage conversation with AI assistant, that has access to the predefined tools, and human.

In [31]:
from langchain_core.runnables import RunnablePassthrough
from langchain.agents import AgentExecutor
from langchain.tools.render import render_text_description
from langchain.agents.format_scratchpad import format_log_to_str

In the prompt template, we include the tools signature and their description.

In [32]:
prompt_chat = prompt.partial(
    tools=render_text_description(list(tools_chat)),
    tool_names=", ".join([t.name for t in tools_chat]),
)

Finally, when we have all components prepared we can define Chat Agent and initialize `AgentExecutor`.

In [33]:
# Setting stop sequence for LLM
llm_with_stop = llm.bind(stop=["}\n"])

agent_chat = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
    )
    | prompt_chat
    | llm_with_stop
    | CustomJSONAgentOutputParser()
)

agent_executor_chat = AgentExecutor(
    agent=agent_chat, tools=tools_chat, verbose=True, handle_parsing_errors=True, return_intermediate_steps=True
)

Notice that we use `handle_parsing_errors=True` to ensure that when LLM is not able to determine what step to take because its output does not have the correct structure to be handled  by output parser, the user's input will be sent back to the model with the appropriate information to improve the model reponse. Setitng this parameter to True help to keep chat conversation smooth, but may also result in multiple requests with many tokens being sent to the model and back.

<a id="chat"></a>
## Structured Chat with memory

To include chat memory in conversation with AI assistant we are going to use `ChatMessageHistory` that simply store chat history in the prompt and send it with every user question to the LLM. 

In [34]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [35]:
message_history = ChatMessageHistory()

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor_chat,
    get_session_history=lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [36]:
import time

In [37]:
def agentExecutorWithExecTime(input_text: str, session_id: str) -> dict:
    start_time = time.time()
    answer = agent_with_chat_history.invoke(
        {"input": input_text},
        config={"configurable": {"session_id": session_id}},
    )
    end_time = time.time()
    execution_time = end_time - start_time
    return {
        "input": answer['input'],
        "chat_history": answer['chat_history'],
        "output": answer['output'],
        "execution_time": f'{round(execution_time, 2)} sec'
    }

In [38]:
agentExecutorWithExecTime("Which product did Alice buy?", "wx")



> Entering new AgentExecutor chain...

```
{
  "action": "sql_db_list_tables",
  "action_input": ""
}

```
Observation:customer_purchase_history, product_inventory, sales_data
```
{
  "action": "sql_db_schema",
  "action_input": "customer_purchase_history, product_inventory, sales_data"
}

```
Observation:
CREATE TABLE customer_purchase_history (
	customerid SMALLINT, 
	customername VARCHAR(7), 
	productid VARCHAR(4), 
	purchasedate DATE, 
	quantity SMALLINT
)

/*
3 rows from customer_purchase_history table:
customerid	customername	productid	purchasedate	quantity
101	Alice	P001	2023-06-01	2
102	Bob	P002	2023-06-02	1
103	Charlie	P003	2023-06-03	1
*/


CREATE TABLE product_inventory (
	productid VARCHAR(4), 
	productname VARCHAR(7), 
	category VARCHAR(11), 
	stock SMALLINT, 
	price DECIMAL(8, 2)
)

/*
3 rows from product_inventory table:
productid	productname	category	stock	price
P001	iPhone	Phones	50	999.99
P002	iPad	Tablets	100	329.99
P003	Mac	Computers	30	1199.99
*/


CREATE TABLE s

{'input': 'Which product did Alice buy?',
 'chat_history': [],
 'output': 'Alice bought iPhone and AirPods.',
 'execution_time': '11.18 sec'}

In [39]:
agentExecutorWithExecTime("What is her reaction to those products?", "wx")



> Entering new AgentExecutor chain...

```
{
  "action": "milvus_retriever",
  "action_input": "Alice's reaction to iPhone and AirPods"
}

```
Observation:[Document]
MentionID: M003
Text: "Just got the new Mac, and it's amazing! #apple #computers"
ProductID: P003
Platform: Instagram
Name: Charlie

MentionID: M004
Text: "The AirPods have great sound quality. #apple #accessories"
ProductID: P004
Platform: Instagram
Name: Alice

MentionID: M005
Text: "The new iPhone is impressive. Can't wait to explore all its features. #apple"
ProductID: P001
Platform: Twitter
Name: David

MentionID: M006
Text: "iPad is my go-to device for reading and browsing. #apple #ipad"
ProductID: P002
Platform: Facebook
Name: Frank

MentionID: M007
Text: "My new Mac is a powerhouse. Perfect for my design work. #apple #mac"
ProductID: P003
Platform: Instagram
Name: Charlie

MentionID: M008
Text: "The AirPods' noise cancellation is top-notch. #apple #airpods"
ProductID: P004
Platform: Twitter
Name: Alice

MentionID

{'input': 'What is her reaction to those products?',
 'chat_history': [HumanMessage(content='Which product did Alice buy?'),
  AIMessage(content='Alice bought iPhone and AirPods.')],
 'output': "Alice's reaction to the iPhone is positive, stating that it is fantastic with great build quality and very useful. However, her reaction to the AirPods is negative, mentioning that they are not as good as she expected and broke after a week.",
 'execution_time': '4.76 sec'}